<a href="https://colab.research.google.com/github/seilsepour/SSTSA/blob/main/Labelling_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/drive/1tUr5t0ZJ-I4Ni40dkbjku92HAU5SyR_2?usp=sharing#scrollTo=wK7GAJI79bAZ

In [ ]:
!pip install sentistrength

In [ ]:
!pip install pycm

In [ ]:
!pip install afinn

In [ ]:
!pip install --upgrade gensim


In [ ]:
!pip install -q vaderSentiment
!pip install -q textblob
!pip install -q flair

# Configuration

In [ ]:
import numpy as np
import pandas as pd

import string
from __future__ import print_function

import gensim
from scipy import spatial

from sklearn.metrics import accuracy_score

import re

In [ ]:
d2vModelModel = '/MYDRIVE/My Drive/Colab Notebooks/SSTSA/data/d2vModelPangLee-00-11-04.model'
d2vModelTxt = '/MYDRIVE/My Drive/Colab Notebooks/SSTSA/data/d2vModelPangLee-00-11-04.txt'

w2vMovieTxt = '/MYDRIVE/My Drive/Colab Notebooks/SSTSA/data/w2vBook-01-02-17.txt'
w2vMovieM = '/MYDRIVE/My Drive/Colab Notebooks/SSTSA/data/w2vBook-01-02-17.h5'
#pangLeeDataset = '/MYDRIVE/My Drive/Colab Notebooks/SecondPaper/data/Preprocessed-Opinion-Words.csv'
#running again to compute just some columns
pangLeeDataset = '/MYDRIVE/My Drive/Colab Notebooks/SSTSA/data/MR02-Opinion-Words-ALL.xlsx'

In [ ]:
#Making Word2vec
embeddingDim = 300
maxLen = embeddingDim
testSize=0.2
windowSize=5
minCount=1
iterCount=10
sG = 0 #skip gram wil be used


#Making Model
validationSplit = 0.2
#maxFeatures = numWords
maxLen = 500
isTrainable = False
batchSize = 1000


# Colab

In [ ]:
#No need in local
from google.colab import drive
drive.mount('/MYDRIVE', force_remount=True)

# Common

https://radimrehurek.com/gensim/auto_examples/tutorials/run_wmd.html

In [ ]:
#especially for word mover
from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.
stop_words = stopwords.words('english')

def remove_pronoun(tokens):
        pronouns = ['i', 'you', 'he', 'she', 'it', 'we', 'they', 'what', 'who','me', 'him', 'her', 'it', 'us', 'you', 'them', 'whom','mine', 'yours', 'his', 'hers', 'ours', 'theirs','this', 'that', 'these', 'those']

        for token in tokens:
            for pronoun in pronouns:
                if token == pronoun:
                    #print("^^^^^^^^\:   ", token, pronoun)
                    tokens.remove(token)
                    break


        return tokens

def preprocess(sentence):
            user_review = sentence
            #user_review = self.split_review_text(user_review)
            user_review = user_review.strip().replace("\n", " ").replace("\r", " ")
            user_review = user_review.strip().replace(".", ".").replace(".", ".")
            user_review = user_review.lower()
            user_review =  user_review.replace(",", " ")
            user_review =  user_review.replace("'", " ")
            user_review =  user_review.replace(".", " ")
            user_review =  user_review.replace("-", " ")
            user_review =  user_review.replace("(", " ")
            user_review =  user_review.replace(")", " ")
            user_review =  user_review.replace("<", " ")
            user_review =  user_review.replace(">", " ")
            user_review = ''.join([i for i in user_review if not i.isdigit()])
            user_review = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',user_review)
            user_review = re.sub(r'#([^\s]+)', r'\1', user_review)
            """ Removes unicode strings like "\u002c" and "x96" """
            user_review = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', user_review)
            user_review = re.sub(r'[^\x00-\x7f]',r'',user_review)
            user_review = re.sub(r"(\?)\1+", '', user_review)
            user_review = re.sub(r"(\.)\1+", '', user_review)
            user_review = re.sub(r"(\!)\1+", '', user_review)
            return [w for w in user_review.lower().split() if w not in stop_words]


def preprocess2(row):
            user_review = row[1]
            #user_review = self.split_review_text(user_review)
            user_review = user_review.strip().replace("\n", " ").replace("\r", " ")
            user_review = user_review.strip().replace(".", ".").replace(".", ".")
            user_review = user_review.lower()
            user_review =  user_review.replace(",", " ")
            user_review =  user_review.replace("'", " ")
            user_review =  user_review.replace(".", " ")
            user_review =  user_review.replace("-", " ")
            user_review =  user_review.replace("(", " ")
            user_review =  user_review.replace(")", " ")
            user_review =  user_review.replace("<", " ")
            user_review =  user_review.replace(">", " ")
            user_review = ''.join([i for i in user_review if not i.isdigit()])
            user_review = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',user_review)
            user_review = re.sub(r'#([^\s]+)', r'\1', user_review)
            """ Removes unicode strings like "\u002c" and "x96" """
            user_review = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', user_review)
            user_review = re.sub(r'[^\x00-\x7f]',r'',user_review)
            user_review = re.sub(r"(\?)\1+", '', user_review)
            user_review = re.sub(r"(\.)\1+", '', user_review)
            user_review = re.sub(r"(\!)\1+", '', user_review)
            return [w for w in user_review.lower().split() if w not in stop_words]



In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from pycm import ConfusionMatrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier

import gzip
import gensim
import logging


from nltk.sentiment.vader import SentimentIntensityAnalyzer


import nltk
from nltk.corpus.reader import CategorizedPlaintextCorpusReader
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from collections import Counter

import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score


class TF_IDF(object):
    def __init__(self):
        print("..")

    def get_tf_idf(self, X):
        vectorizer = TfidfVectorizer(ngram_range=(1,2), tokenizer=lambda x: x.split())
        X = vectorizer.fit_transform(X)
        #print(X)
        return X


class Performance(object):


    def get_results(self, labels, predictions):
        from sklearn.metrics import confusion_matrix



        conf_matrix = confusion_matrix(labels, predictions)

        #print(conf_matrix)

        precision = np.diag(conf_matrix) / np.sum(conf_matrix, axis = 0)
        recall = np.diag(conf_matrix) / np.sum(conf_matrix, axis = 1)

        accuracy =  np.sum(np.diag(conf_matrix) / np.sum(conf_matrix))

        #print("Total: " , np.sum(conf_matrix))

        #print("Correct/Incorrect : ", np.sum(np.diag(conf_matrix) ),  np.sum(conf_matrix) -  np.sum(np.diag(conf_matrix) ))
        #print("Denominator:", np.sum(conf_matrix, axis = 0))
        #print(np.mean(precision))
        #print(np.mean(recall))

        precision = np.mean(precision)
        recall = np.mean(recall)

        #print("Precision: ", precision)
        #print("Recall: " , recall)


        f1_score = (2 * precision * recall) / (precision + recall)
        #print("F-1 Score:  -----  ", f1_score)
        return conf_matrix, precision,  recall, f1_score,  accuracy


    def calculateMCC(self, labels, predictions):

        cm = ConfusionMatrix(labels, predictions, digit=5)

        #print("\n Kappa-AC1: ", cm.Kappa, cm.AC1)
        #print("\nMCC:")
        #print(cm.MCC)
        return cm.MCC


    def calculate_roc_auc_score(self, labels, predictions):
        score = roc_auc_score(labels, predictions)
        return score
        #print("ROC:" , score)


# Load Data

In [ ]:
dfPangLee = pd.DataFrame()
dfPangLee = pd.read_excel(pangLeeDataset)
#dfPangLee = pd.read_csv(pangLeeDataset, encoding = 'utf-8', header = None)

In [ ]:
dfPangLee.columns

In [ ]:
dfPangLee.head(2)

In [ ]:
#dfPangLee.columns = ['id','ActualText', 'PreprocessedText' ,'Positives', 'Negatives', 'ActualLabel', 'PredictedLabel', 'ConfidenseScore', 'Type']

In [ ]:
dfPangLee.columns = ['id','ActualText' ,'Positives','Positive_Negations', 'Negatives', 'Negative_Negations', 'ActualLabel', 'Type']

In [ ]:
dfPangLee2 = dfPangLee
print(len(dfPangLee2))

print(dfPangLee2.columns)


In [ ]:
targets = dfPangLee2['ActualLabel']

# Labeling

In [ ]:
dfPangLee2.insert(8, "PreProcessed"," ")
dfPangLee2.insert(9, "score_c",0.0)
dfPangLee2.insert(10, "cos_p",0.0)
dfPangLee2.insert(11, "cos_n",0.0)
dfPangLee2.insert(12, "comp_c",0.0)

dfPangLee2.insert(13, "score_dist",0.0) # distance
dfPangLee2.insert(14, "dist_p",0.0)
dfPangLee2.insert(15, "dist_n",0.0)
dfPangLee2.insert(16, "comp_dist",0.0)

dfPangLee2.insert(17, "score_sc",0.0) #soft cosine
dfPangLee2.insert(18, "softcos_p",0.0)
dfPangLee2.insert(19, "softcos_n",0.0)
dfPangLee2.insert(20, "comp_sc",0.0)

dfPangLee2.insert(21, "score_csc",0.0) # cosine + soft cosine
dfPangLee2.insert(22, "csc_p",0.0)
dfPangLee2.insert(23, "csc_n",0.0)
dfPangLee2.insert(24, "comp_csc",0.0)


dfPangLee2.insert(25, "VADER",0.0)
dfPangLee2.insert(26, "comp_VADER",0.0)

dfPangLee2.insert(27, "BLOB",0.0)
dfPangLee2.insert(28, "comp_BLOB",0.0)

dfPangLee2.insert(29, "FLAIR",0.0)
dfPangLee2.insert(30, "comp_FLAIR",0.0)

dfPangLee2.insert(31, "result_positive_minus",0.0)
dfPangLee2.insert(32, "result_negative_minus",0.0)
dfPangLee2.insert(33, "score_overall_minus",0.0)
dfPangLee2.insert(34, "evaluation_minus",0.0)



In [ ]:
dfPangLee2.insert(35, "AFINN",0.0)
dfPangLee2.insert(36, "comp_AFINN",0.0)

In [ ]:
dfPangLee2.insert(37, "MIX",0.0)
dfPangLee2.insert(38, "comp_MIX",0.0)

In [ ]:
dfPangLee2.insert(39, "evaluation_MIX",0.0)

In [ ]:
dfPangLee2.insert(40, "SENTI",0.0)
dfPangLee2.insert(41, "comp_SENTI",0.0)

In [ ]:
dfPangLee2['PreProcessed'] = dfPangLee2.apply (lambda row: preprocess2(row), axis=1)

# Make Doc2Vec and SAVE

In [ ]:
def prepare_data(dfTemp, tokens_only=False):
    for index, row in dfTemp.iterrows():
            #tokens = gensim.utils.simple_preprocess(row[1])
            #tokens = preprocess(row['ActualText'])
            tokens = row['PreProcessed']
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [row[0]])

In [ ]:
dfTrain = dfPangLee2[dfPangLee2['Type'] == 'train']
dfTest = dfPangLee2[dfPangLee2['Type'] == 'test']
print(len(dfTrain))
print(len(dfTest))

In [ ]:
train_corpus = list(prepare_data(dfTrain))
test_corpus = list(prepare_data(dfTest, tokens_only=True))

In [ ]:
len(train_corpus)

In [ ]:
len(test_corpus)

In [ ]:
print(train_corpus[:2])

In [ ]:
print(test_corpus[:2])

In [ ]:
d2vModel = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=40, seed = 1)

In [ ]:
d2vModel.build_vocab(train_corpus)

In [ ]:
#print(f"Word 'penalty' appeared {d2vModel.wv.get_vecator('penalty', 'count')} times in the training corpus.")

In [ ]:
d2vModel.train(train_corpus, total_examples=d2vModel.corpus_count, epochs=d2vModel.epochs)

In [ ]:
d2vModel.save(d2vModelModel)
d2vModel.save(d2vModelTxt)

# Load D2V

In [ ]:
from gensim.models.doc2vec import Doc2Vec
d2vModel = Doc2Vec.load(d2vModelModel)

# Make Word2Vec and SAVE

In [ ]:
reviewMovieLines = list()
for index, row in dfPangLee2.iterrows():
  reviewMovieLines.append( row["PreProcessed"] )
  #print( row[0].split())
  #reviewMovieLines.append( str(row[1]).split() )


print(len(reviewMovieLines))



w2vMovie = gensim.models.Word2Vec( sentences=reviewMovieLines, vector_size= 200, epochs = 10,   window=windowSize, workers=4,
                                  min_count=1,  sg = sG, seed=1 )

w2vGensimModel = gensim.models.Word2Vec()
#vocab size



In [ ]:
print('Vocabulary size: %d' % len(w2vMovie.wv))


w2vMovie.wv.save_word2vec_format( w2vMovieTxt, binary = False)
w2vMovie.save(w2vMovieM)

# Load W2V

In [ ]:
w2vMovie = gensim.models.Word2Vec.load(w2vMovieM)

# JS

https://machinelearningmastery.com/divergence-between-probability-distributions/

In [ ]:
from scipy.spatial.distance import jensenshannon
from numpy import asarray
#js_pq = jensenshannon(p, q, base=2)

In [ ]:
dfPangLee2.columns

# Soft Cosine

https://radimrehurek.com/gensim//auto_examples/tutorials/run_scm.html

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import SparseTermSimilarityMatrix, WordEmbeddingSimilarityIndex
#from gensim.similarities import WordEmbeddingSimilarityIndex


documents = list()
for index, row in dfPangLee2.iterrows():
  documents.append( row['PreProcessed'] )

dictionary = Dictionary(documents)

documents2 = list()
for document in documents:
  documents2.append(dictionary.doc2bow(document))

import gensim.downloader as api
#model = api.load('word2vec-google-news-300')


tfidf = TfidfModel(documents2)
termsim_index = WordEmbeddingSimilarityIndex(w2vMovie.wv)
#termsim_index = WordEmbeddingSimilarityIndex(model)

termsim_matrix = SparseTermSimilarityMatrix(termsim_index, dictionary, tfidf)

In [ ]:
def Calculate_SoftCosine_Similarity(sent1,sent2):
  sent1 = preprocess(sent1)
  sent2 = preprocess(sent2)
  sent1 = dictionary.doc2bow(sent1)
  sent2 = dictionary.doc2bow(sent2)
  sent1 = tfidf[sent1]
  sent2 = tfidf[sent2]
  similarity = termsim_matrix.inner_product(sent1, sent2, normalized=(True, True))
  return similarity

In [ ]:
#dfPangLee2 = dfPangLee
#dfPangLee2 = dfPangLee2[dfPangLee2["id"] < 10 ]
for index, row in dfPangLee2.iterrows():
    #text = gensim.utils.simple_preprocess(row[1])
    text = row["PreProcessed"]

    positives = row["Positives"].replace('[','').replace(']','').replace("'",'').replace(' ','').split(',')
    negatives = row["Negatives"].replace('[','').replace(']','').replace("'",'').replace(' ','').split(',')


    positive_negations = row["Positive_Negations"].replace('[','').replace(']','').replace("'",'').replace(' ','').split(',')
    negative_negations = row["Negative_Negations"].replace('[','').replace(']','').replace("'",'').replace(' ','').split(',')



    sc_p = 0
    sc_n = 0
    for pos in positives:
        sc_p =  sc_p + Calculate_SoftCosine_Similarity(row['ActualText'],pos)




    for neg in negatives:
        sc_n =  sc_n + Calculate_SoftCosine_Similarity(row['ActualText'],neg)


    #Adding negations
    for pos in positive_negations:
        sc_p =  sc_p - Calculate_SoftCosine_Similarity(row['ActualText'],pos)




    for neg in negative_negations:
        sc_n =  sc_n - Calculate_SoftCosine_Similarity(row['ActualText'],neg)



    positives2 = positives
    negatives2 = negatives
    positives2 = ' '.join(dict.fromkeys(positives2))
    negatives2 = ' '.join(dict.fromkeys(negatives2))

    #print(positives2)
    #print(negatives2)



    sc_p =  sc_p + Calculate_SoftCosine_Similarity(row['ActualText'],positives2)
    sc_n =  sc_n + Calculate_SoftCosine_Similarity(row['ActualText'],negatives2)
    #print(sc_p)
    #print(sc_n)

    dfPangLee2.loc[index, 'softcos_p'] = sc_p
    dfPangLee2.loc[index, 'softcos_n'] = sc_n

    if sc_p > sc_n:
         dfPangLee2.loc[index, 'score_sc'] = 1
         if row["ActualLabel"] == 1:
           dfPangLee2.loc[index, 'comp_sc'] = "match"
         else:
            dfPangLee2.loc[index, 'comp_sc'] = "not match"

    else:
         dfPangLee2.loc[index, 'score_sc'] = 0
         if row["ActualLabel"] == 0:
           dfPangLee2.loc[index, 'comp_sc'] = "match"
         else:
            dfPangLee2.loc[index, 'comp_sc'] = "not match"


In [ ]:
print(len(dfPangLee2[dfPangLee2["comp_sc"] == "not match" ]))

# Cosine

In [ ]:
#dfPangLee2 = dfPangLee2[dfPangLee2["id"] < 1 ]
for index, row in dfPangLee2.iterrows():
    #text = gensim.utils.simple_preprocess(row[1])

    #text = preprocess(row['ActualText'])
    text = row["PreProcessed"]
    #print(text)
    vec_p =  np.zeros(300)
    vec_n =  np.zeros(300)
    vec_main = d2vModel.infer_vector(text)
    positives = row['Positives'].replace('[','').replace(']','').replace("'",'').replace(' ','').split(',')
    #print(positives)
    negatives = row['Negatives'].replace('[','').replace(']','').replace("'",'').replace(' ','').split(',')
    #print(negatives)
    positive_negations = row['Positive_Negations'].replace('[','').replace(']','').replace("'",'').replace(' ','').split(',')

    negative_negations = row['Negative_Negations'].replace('[','').replace(']','').replace("'",'').replace(' ','').split(',')



    cos_p = 0
    cos_n = 0
    for pos in positives:
        vec_p =  d2vModel.infer_vector([pos])
        #print(vec_p)
        cos_p = cos_p + spatial.distance.cosine(vec_main, vec_p)
        #print(cos_p)



    for neg in negatives:
        vec_n =  d2vModel.infer_vector([neg])
        #print(vec_n)
        cos_n = cos_n + spatial.distance.cosine(vec_main, vec_n)
        #print(cos_n)

    vec_p =  d2vModel.infer_vector(positives)
    vec_n =  d2vModel.infer_vector(negatives)
    cos_p = cos_p + spatial.distance.cosine(vec_main, vec_p)
    cos_n = cos_n + spatial.distance.cosine(vec_main, vec_n)


    #Adding Negations
    for pos in positive_negations:
        vec_p =  d2vModel.infer_vector([pos])
        #print(vec_p)
        cos_p = cos_p - spatial.distance.cosine(vec_main, vec_p)
        #print(cos_p)



    for neg in negative_negations:
        vec_n =  d2vModel.infer_vector([neg])
        #print(vec_n)
        cos_n = cos_n - spatial.distance.cosine(vec_main, vec_n)
        #print(cos_n)

    #print(cos_p)
    #print(cos_n)
    if cos_p < cos_n:
            dfPangLee2.loc[index, 'score_c'] = 0
            if row["ActualLabel"] == 0:
                 dfPangLee2.loc[index, 'comp_c'] = "match"
            else:
                 dfPangLee2.loc[index, 'comp_c'] = "not match"
    else:
            dfPangLee2.loc[index, 'score_c'] = 1
            if row["ActualLabel"] == 1:
               dfPangLee2.loc[index, 'comp_c'] = "match"
            else:
               dfPangLee2.loc[index, 'comp_c'] = "not match"

    dfPangLee2.loc[index, 'cos_p'] = cos_p
    dfPangLee2.loc[index, 'cos_n'] = cos_n



# Distance

In [ ]:
def Calculate_Word_Distance(sent1,sent2):
    sent1 = preprocess(sent1)
    sent2 = preprocess(sent2)
    distance = abs (w2vMovie.wv.wmdistance(sent1, sent2))
    #print('distance = %.4f' % distance)
    return distance

In [ ]:
#dfPangLee2 = dfPangLee2[dfPangLee2["id"] < 3 ]
for index, row in dfPangLee2.iterrows():
    #text = preprocess(row['ActualText'])
    text = row["PreProcessed"]


    positives = row["Positives"].replace('[','').replace(']','').replace("'",'').replace(' ','').split(',')
    negatives = row["Negatives"].replace('[','').replace(']','').replace("'",'').replace(' ','').split(',')

    positive_negations = row["Positive_Negations"].replace('[','').replace(']','').replace("'",'').replace(' ','').split(',')
    negative_negations = row["Negative_Negations"].replace('[','').replace(']','').replace("'",'').replace(' ','').split(',')

    #Mean distance
    distance_p = 0
    distance_n = 0

    positives2 = list(dict.fromkeys(positives))
    negatives2 = list(dict.fromkeys(negatives))

    positive_negations2 = list(dict.fromkeys(positive_negations))
    negative_negations2 = list(dict.fromkeys(negative_negations))


    #positives2 = positives
    #negatives2 = negatives
    positives2 = ' '.join(dict.fromkeys(positives2))
    negatives2 = ' '.join(dict.fromkeys(negatives2))

    positive_negations2 = ' '.join(dict.fromkeys(positive_negations2))
    negative_negations2 = ' '.join(dict.fromkeys(negative_negations2))


    #print(positives2)
    #print(negatives2)

    distance_p =  Calculate_Word_Distance(row['ActualText'],positives2)
    if len(positive_negations2) > 0:
       distance_p = distance_p -  Calculate_Word_Distance(row['ActualText'], positive_negations2)
    #print(distance_p)
    distance_n =  Calculate_Word_Distance(row['ActualText'],negatives2)
    if len(negative_negations2) > 0:
        distance_n =  distance_n - Calculate_Word_Distance(row['ActualText'], negative_negations2)

    #print(distance_n)

    dfPangLee2.loc[index, 'dist_p'] = distance_p
    dfPangLee2.loc[index, 'dist_n'] = distance_n

    if distance_p < distance_n:
      dfPangLee2.loc[index, 'score_dist'] = 1
      if row["ActualLabel"] == 1:
           dfPangLee2.loc[index, 'comp_dist'] = "match"
      else:
            dfPangLee2.loc[index, 'comp_dist'] = "not match"
    elif  distance_p > distance_n:
      dfPangLee2.loc[index, 'score_dist'] = 0
      if row["ActualLabel"] == 0:
           dfPangLee2.loc[index, 'comp_dist'] = "match"
      else:
            dfPangLee2.loc[index, 'comp_dist'] = "not match"


In [ ]:
print(len(dfPangLee2[dfPangLee2["comp_dist"] == "not match" ]))

#  MINUS and DIVIDE

In [ ]:
print(len(dfPangLee2[dfPangLee2["comp_c"] == "not match" ]))

In [ ]:
print(len(dfPangLee2[dfPangLee2["comp_dist"] == "not match" ]))

In [ ]:
print(len(dfPangLee2[dfPangLee2["comp_sc"] == "not match" ]))

In [ ]:
print(len(dfPangLee2[dfPangLee2["comp_sc"] == "not match" ]))

In [ ]:
def add_result_positive_minus (row):
  return (row['cos_p'] + row['softcos_p']) - row['dist_p']


def add_result_negative_minus (row):
  return (row['cos_n'] + row['softcos_n']) - row['dist_n']


def add_result_total_minus (row):
  if row['result_positive_minus'] > row['result_negative_minus']:
     return 1
  else:
    return 0

def add_result_evaluation_minus (row):
  if row['score_overall_minus'] == row['ActualLabel']:
    return "match"
  else:
    return "not match"


In [ ]:
dfPangLee2['result_positive_minus'] = dfPangLee2.apply (lambda row: add_result_positive_minus(row), axis=1)
dfPangLee2['result_negative_minus'] = dfPangLee2.apply (lambda row: add_result_negative_minus(row), axis=1)
dfPangLee2['score_overall_minus'] = dfPangLee2.apply (lambda row: add_result_total_minus(row), axis=1)
dfPangLee2['evaluation_minus'] = dfPangLee2.apply (lambda row: add_result_evaluation_minus(row), axis=1)

In [ ]:
print(len(dfPangLee2[dfPangLee2["evaluation_minus"] == "not match" ]))

In [ ]:
print(len(dfPangLee2[dfPangLee2["evaluation_minus"] == "match" ]))

In [ ]:
def add_result_positive_scs (row):
  return (row['cos_p'] + row['softcos_p'])


def add_result_negative_scs (row):
  return (row['cos_n'] + row['softcos_n'])


def add_result_total_scs (row):
  if row['csc_p'] > row['csc_n']:
     return 1
  else:
    return 0

def add_result_evaluation_scs (row):
  if row['score_csc'] == row['ActualLabel']:
    return "match"
  else:
    return "not match"

dfPangLee2['csc_p'] = dfPangLee2.apply (lambda row: add_result_positive_scs(row), axis=1)
dfPangLee2['csc_n'] = dfPangLee2.apply (lambda row: add_result_negative_scs(row), axis=1)
dfPangLee2['score_csc'] = dfPangLee2.apply (lambda row: add_result_total_scs(row), axis=1)
dfPangLee2['comp_csc'] = dfPangLee2.apply (lambda row: add_result_evaluation_scs(row), axis=1)

In [ ]:
print(len(dfPangLee2[dfPangLee2["comp_csc"] == "not match" ]))

In [ ]:
print(len(dfPangLee2[dfPangLee2["comp_csc"] == "match" ]))

# MIX

In [ ]:
def add_result_mix (row):
  #score = ((row['cos_p'] + row['softcos_p']) - (row['cos_n'] + row['softcos_n']))/(row['dist_p'] + row['dist_n'])
  #score = ((row['cos_p'] + row['softcos_p']) - row['dist_p']) - ((row['cos_n'] + row['softcos_n'])-row['dist_n'] )
  #score = ((row['cos_p'] + row['softcos_p'])/row['dist_p'])  - ((row['cos_n'] + row['softcos_n'])/row['dist_n'])
  #score2 = row['dist_p'] - row['dist_n']
  if ((row['cos_p'] + row['softcos_p'])> (row['cos_n'] + row['softcos_n'])) and (row['dist_p'] < row['dist_n'] ):
    score = 1
  elif ((row['cos_p'] + row['softcos_p'])< (row['cos_n'] + row['softcos_n'])) and (row['dist_p'] > row['dist_n'] ):
    score = 0
  else:
    if row['dist_p'] == 0 or row['dist_p'] == "inf":
      score = 1
    elif row['dist_n'] == 0 or row['dist_n'] == "inf":
      score = 0
    else:
          temp = ((row['cos_p'] + row['softcos_p'])/row['dist_p'])  - ((row['cos_n'] + row['softcos_n'])/row['dist_n'])
          if temp >0:
            score = 1
          else:
            score = 0
  return score

def mix_eval (row):
  if row['MIX'] == row['ActualLabel']:
    return "match"
  else:
    return "not match"

In [ ]:
row

In [ ]:
dfPangLee2['MIX'] = dfPangLee2.apply (lambda row: add_result_mix (row), axis=1)
predictions = dfPangLee2.apply (lambda row: add_result_mix (row), axis=1)
dfPangLee2['evaluation_MIX'] = dfPangLee2.apply (lambda row: mix_eval(row), axis=1)

In [ ]:
print(len(dfPangLee2[dfPangLee2["MIX"] ==0 ]))

In [ ]:
print(len(dfPangLee2[dfPangLee2["MIX"] ==1 ]))

In [ ]:
print(len(dfPangLee2[dfPangLee2["evaluation_MIX"] == "not match" ]))

In [ ]:
accuracy_score(predictions,targets)

# Save

In [ ]:
#dfPangLee2.to_csv('/MYDRIVE/My Drive/Colab Notebooks/SSTSA/data/resultAll-dvd-01-02-20.csv', index=False)

# VADER

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

def eval_vader (row):
  if row['VADER'] == row['ActualLabel']:
    return "match"
  else:
    return "not match"

def text_sentiment_vader(row):
 vs = analyzer.polarity_scores(row["ActualText"])
 return int(vs.get("compound")>0)


In [ ]:
 predictions = dfPangLee2.apply (lambda row: text_sentiment_vader(row), axis=1)
 dfPangLee2['VADER'] = dfPangLee2.apply (lambda row: text_sentiment_vader(row), axis=1)
 dfPangLee2['comp_VADER'] = dfPangLee2.apply (lambda row: eval_vader(row), axis=1)
 print(predictions)

In [ ]:
accuracy_score( predictions.values  , targets )

In [ ]:
print(len(dfPangLee2[dfPangLee2["comp_VADER"] == "not match" ]))

# TEXTBLOB

In [ ]:
dfPangLee2.columns

In [ ]:
from textblob import TextBlob

def eval_blob (row):
  if row['BLOB'] == row['ActualLabel']:
    return "match"
  else:
    return "not match"

def text_sentiment_blob(row):
    testimonial = TextBlob(row["ActualText"])
    score = testimonial.sentiment.polarity
    if score > 0:
      return 1
    else:
      return 0


In [ ]:
predictions = dfPangLee2.apply (lambda row: text_sentiment_blob(row), axis=1)
dfPangLee2['BLOB'] = predictions
#dfPangLee2['BLOB'] = dfPangLee2.apply (lambda row: text_sentiment_blob(row), axis=1)
dfPangLee2['comp_BLOB'] = dfPangLee2.apply (lambda row: eval_blob(row), axis=1)
#accuracy_score(predictions,targets)

In [ ]:
accuracy_score(predictions,targets)

In [ ]:
print(len(dfPangLee2[dfPangLee2["comp_BLOB"] == "not match" ]))

In [ ]:
analyzer.polarity_scores("The food as great!")

# FLAIR

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence
classifierFlair = TextClassifier.load('en-sentiment')

def eval_flair (row):
  if row['FLAIR'] == row['ActualLabel']:
    return "match"
  else:
    return "not match"

def text_sentiment_flair(row):
  sentence = Sentence(row["ActualText"])
  classifierFlair.predict(sentence)
  if sentence.labels[0].value == "POSITIVE":
    return 1
  else:
    return 0



In [ ]:
predictions = dfPangLee2.apply (lambda row: text_sentiment_flair(row), axis=1)
dfPangLee2['FLAIR'] = predictions
#dfPangLee2['FLAIR'] = dfPangLee2.apply (lambda row: text_sentiment_flair(row), axis=1)
dfPangLee2['comp_FLAIR'] = dfPangLee2.apply (lambda row: eval_flair(row), axis=1)

In [ ]:
accuracy_score(predictions.values,targets)

In [ ]:
print(len(dfPangLee2[dfPangLee2["comp_FLAIR"] == "not match" ]))

# AFINN

In [ ]:
from afinn import Afinn
afn = Afinn()
def eval_afinn (row):
  if row['FLAIR'] == row['ActualLabel']:
    return "match"
  else:
    return "not match"

def text_sentiment_afinn(row):
  score = afn.score(row["ActualText"])
  if score > 0:
    return 1
  elif score < 0:
    return 0
  else:
      return score



predictions = dfPangLee2.apply (lambda row: text_sentiment_afinn(row), axis=1)
dfPangLee2['AFINN'] = dfPangLee2.apply (lambda row: text_sentiment_afinn(row), axis=1)
dfPangLee2['comp_AFINN'] = dfPangLee2.apply (lambda row: eval_afinn(row), axis=1)

In [ ]:
accuracy_score( predictions , targets )

In [ ]:
print(len(dfPangLee2[dfPangLee2["comp_AFINN"] == "not match" ]))

In [ ]:
print(len(dfPangLee2[dfPangLee2["comp_AFINN"] == "match" ]))

In [ ]:
print(len(dfPangLee2[dfPangLee2["comp_AFINN"] == "match" ]))

# SentiStrength

https://github.com/zhunhung/Python-SentiStrength

In [ ]:
pip install sentistrength

In [ ]:
from sentistrength import PySentiStr


def eval_senti (row):
  if row['SENTI'] == row['ActualLabel']:
    return "match"
  else:
    return "not match"

def text_sentiment_senti(row):
  score = senti.getSentiment(row["ActualText"], score="binary")
  #print(score[0])
  if score[0] >= 0:
    return 1
  else:
    return 0



In [ ]:
senti = PySentiStr()
senti.setSentiStrengthPath('/MYDRIVE/My Drive/Colab Notebooks/SSTSA/data/SentiStrengthCom.jar') # Note: Provide absolute path instead of relative path
senti.setSentiStrengthLanguageFolderPath('/MYDRIVE/My Drive/Colab Notebooks/SSTSA/data/SentiStrengthData/') # Note: Provide absolute path instead of relative path
score = senti.getSentiment("I hate you")
print(score[0])


In [ ]:


predictions = dfPangLee2.apply (lambda row: text_sentiment_senti(row), axis=1)
dfPangLee2['SENTI'] = predictions
#dfPangLee2['SENTI'] = dfPangLee2.apply (lambda row: text_sentiment_senti(row), axis=1)
dfPangLee2['comp_SENTI'] = dfPangLee2.apply (lambda row: eval_senti(row), axis=1)

In [ ]:
accuracy_score( predictions , targets )

# SAVE

In [ ]:
dfPangLee2.to_csv('/MYDRIVE/My Drive/Colab Notebooks/SSTSA/data/resultAll-MR-01-02-22.csv', index=False)

In [ ]:
dfPangLee2.head()

# Accuracy Score

## Minus

In [ ]:
dfPangLee2.columns

In [ ]:
predictions = dfPangLee2["score_overall_minus"]

In [ ]:
accuracy_score(predictions,targets)

In [ ]:
print(len(dfPangLee2[dfPangLee2["evaluation_minus"] == "not match" ]))

## csc

In [ ]:
predictions = dfPangLee2["score_csc"]

print(accuracy_score(predictions,targets))

In [ ]:
accuracy_score(predictions,targets)

In [ ]:
print(len(dfPangLee2[dfPangLee2["comp_csc"] == "not match" ]))

## ِDist

In [ ]:
predictions = dfPangLee2["score_dist"]

print(accuracy_score(predictions,targets))

## cosine

In [ ]:
predictions = dfPangLee2["score_c"]

print(accuracy_score(predictions,targets))

## soft cosine

In [ ]:
predictions = dfPangLee2["score_sc"]

print(accuracy_score(predictions,targets))